In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import sys
from glob import glob

import math
import random 

import cv2                                 # image를 읽기 위한 open cv library
import xml.etree.ElementTree as et         # xml 파일을 parsing 하기 위한 library
from matplotlib.patches import Rectangle   # Bounding box를 그리기 위함

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations

import albumentations as A    # CoarseDropout 인지 안 된다는 문제 발생 -> pip install 아래 세 가지 코드 실행 필요

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style('whitegrid')

drive_project_root = 'drive/MyDrive/data/'
sys.path.append(drive_project_root)

image_root = 'drive/MyDrive/data/images/'
anno_root = 'drive/MyDrive/data/annotations/'

image_dir = image_root
bbox_dir = anno_root + 'xmls/'    # bounding box
seg_dir = anno_root + 'trimaps/'  # segmentation map

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

!ls

In [ ]:
pip install git+https://github.com/albumentations-team/albumentations.git  # Data Agumentation

In [ ]:
pip uninstall opencv-python

In [ ]:
pip install opencv-python

---
# 1. Data Augmentation
- 보통 training단계에서 많이 사용됨
- 데이터 양을 늘리기 위해 원본 이미지를 여러 가지 방식으로 변환하는 것
- 결함이 있는 데이터에 대해서도 robust한 모델을 만들기 위함
- 3rd party library 사용
  - 많은 사람들이 사용하는 것 추천 - Albumentations : made by kaggle masters

In [ ]:
class Augmentation:
    def __init__(self, size, mode='train'):
        if mode == 'train':
            # Declare an augmentation pipeline
            # 여러 번 호출할 예정이라 transform이 아닌 self.transform으로 저장함
            self.transform = A.Compose([
                # A.RandomCrop(width=256, height=256),  # resize: 외부에서 따로 할 예정
                # 좌우반전 (test 이미지에서 상하반전은 별로 없지만 좌우반전된 이미지는 많이 있을 것 같아서 추가함)
                # p : 해당 변화를 적용되는 정도 (확률)
                A.HorizontalFlip(p=0.5),
                A.ShiftScaleRotate(
                    p=0.5,
                    shift_limit=0.05,  # 이미지 상하좌우 이동 (가로 길이가 최대 몇% 넘어가도 되는 지)
                    scale_limit=0.05,  # 이미지 확대/축소 (최대 5%)
                    rotate_limit=15,   # 이미지 회전 (최대 15도)
                ),
                A.CoarseDropout(                 # 이미지에 구멍 뚫기
                    p=0.5,
                    max_holes=8,                 # 최대 구멍 개수
                    max_height=int(0.1 * size),  # 가로 최대 길이 : 이미지의 10%
                    max_width=int(0.1 * size),
                ),
                A.RandomBrightnessContrast(p=0.2),
            ])

    def __call__(self, **kwargs):
        if self.transform:   # train mode인 경우
            augmented = self.transform(**kwargs)
            img = augmented['image']
            return img

---
# 2. Data Loader

In [ ]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, batch_size, csv_path, image_size,
                 fold, mode='train', shuffle=True):
        self.batch_size = batch_size
        self.image_size = image_size
        self.shuffle = shuffle
        self.fold = fold
        self.mode = mode
        
        self.df = pd.read_csv(csv_path)
        
        if self.mode == 'train':
            self.df = self.df[self.df['fold'] != self.fold]
        elif self.mode == 'val':
            self.df = self.df[self.df['fold'] == self.fold]
        
        #### Remove invalid files
        #### https://github.com/tensorflow/models/issues/3134
        invalid_filenames = [
            'Egyptian_Mau_14',
            'Egyptian_Mau_139',
            'Egyptian_Mau_145',
            'Egyptian_Mau_156',
            'Egyptian_Mau_167',
            'Egyptian_Mau_177',
            'Egyptian_Mau_186',
            'Egyptian_Mau_191',
            'Abyssinian_5',
            'Abyssinian_34',
            'chihuahua_121',
            'beagle_116'
        ]
        self.df = self.df[~self.df['file_name']. \
                          isin(invalid_filenames)]

        self.transform = Augmentation(image_size, mode)  ### Augmentation 적용

        self.on_epoch_end()
            
    def __len__(self):
        return math.ceil(len(self.df) / self.batch_size)
        
    def __getitem__(self, idx):
        strt = idx * self.batch_size
        fin = (idx + 1) * self.batch_size
        data = self.df.iloc[strt:fin]
        
        batch_x, batch_y = self.get_data(data)
        
        return np.array(batch_x), np.array(batch_y)
        
    def get_data(self, data):
        batch_x = []
        batch_y = []
        
        for _, r in data.iterrows():
            file_name = r['file_name']
            
            image = cv2.imread(f'{image_root}{file_name}.jpg')
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, (self.image_size, self.image_size))

            if self.mode == 'train':   ### Augmentation 적용
                # augmentation 중에는 image가 uint8인 경우에만 적용되는 것들이 있음
                # -> image를 먼저 uint8로 변환하기
                image = image.astype('uint8')
                image = self.transform(image=image)

            image = image.astype('float32')
            image = image / 255.
            
            label = int(r['species']) - 1
            
            batch_x.append(image)
            batch_y.append(label)
        
        return batch_x, batch_y

    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)

In [ ]:
csv_path = drive_project_root+'kfolds.csv'

train_generator = DataGenerator(
    fold=1,
    mode='train',
    csv_path=csv_path,
    batch_size=128,
    image_size=256,
    shuffle=True
)

valid_generator = DataGenerator(
    fold=1,
    mode='val',
    csv_path=csv_path,
    batch_size=128,
    image_size=256,
    shuffle=True
)

- CoarseDropout이 적용된 결과 0.5%의 확률로 특정 이미지에 hole들이 뚫려있음
- 랜덤하게 적용되기 때문에 아래 코드 다시 돌려보면 CoarseDropout 적용된 이미지가 계속 달라짐
- 회전된 이미지도 볼 수 있음

In [ ]:
class_name = ['Cat', 'Dog']

for batch in train_generator:
    X, y = batch
    plt.figure(figsize=(10,10))

    for i in range(9):
        ax = plt.subplot(3, 3, i+1)
        plt.imshow(X[i])
        plt.title(class_name[y[i]])
        plt.axis('off')
    break

---
# 3. Modeling

In [ ]:
def get_sequential_model(input_shape):
    model = keras.Sequential(
        [
            # Input
            layers.Input(input_shape),

            # 1st Conv block
            layers.Conv2D(64, 3, strides=1, activation='relu', padding='same'),
            layers.Conv2D(64, 3, strides=1, activation='relu', padding='same'),
            layers.MaxPool2D(),
            layers.BatchNormalization(),
            layers.Dropout(0.5),

            # 2nd Conv block
            layers.Conv2D(128, 3, strides=1, activation='relu', padding='same'),
            layers.Conv2D(128, 3, strides=1, activation='relu', padding='same'),
            layers.MaxPool2D(),
            layers.BatchNormalization(),
            layers.Dropout(0.3),
        
            # Classfier
            layers.GlobalMaxPool2D(),
            layers.Dense(128, activation='relu'),
            layers.Dense(1, activation='sigmoid')
        ]
    )

    return model

# input_shape = (256, 256, 3)
input_shape = (16, 16, 3)    # memory error 때문에 input size 바꿔봄
model = get_sequential_model(input_shape)

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics='accuracy'
)

model.summary()

---
# 4. Callback functions
- 특정 상황에서 호출하는 함수
  - on_train_begin : 학습이 시작될 때 호출
  - on_epoch_end : 매 epoch이 끝날 때마다 호출
- 자주 쓰이는 callback 함수 3가지 : Early Stopping, Reduce on Plateau,ModelCheckpoint
  - 만약 early stopping (patience: 3)과 reduce on plateau (patience: 10)를 같이 쓰면, 어차피 3일 때 early stopping이 될 것이기 때문에 실제로 reduce on plateau가 불리는 일은 없음

### 4.1 Early Stopping
- 성능이 나아지지 않으면 학습 중단

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    # validation loss가 3 epoch 동안 줄어들지(min) 않으면 학습 멈추기
    monitor='val_loss',
    patience=3,
    verbose=1,
    mode="min",
    restore_best_weights=False
)

### 4.2 Reduce on plateau
- metric 성능이 나아지지 않으면 learning rate를 줄이기

In [ ]:
reduce_on_plateau = tf.keras.callbacks.ReduceLROnPlateau(
    # validation loss이 10 epoch 동안 작아지지 않으면 learning rate를 0.1배로 줄이기
    monitor='val_loss',
    factor=0.1,
    patience=10,
    verbose=1,
    mode='min',
    min_lr=0.001  # 하지만 learning rate가 0.001보다는 작아지면 안됨
)

### 4.3 Model Checkpoint

In [ ]:
filepath = '{drive_project_root}{epoch:02d}-{val_loss:.2f}.hdf5'
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath,
    monitor='val_loss',  # validation loss가 작아질 때마다 저장
    verbose=1,
    save_best_only=True,     # best 외 불필요한 건 저장X
    save_weights_only=False, # weight 파일만 따로 저장할지
    mode='min'
)

---
# 5. Model fitting

In [ ]:
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=10,
    callbacks=[
        early_stopping,
        reduce_on_plateau,
        model_checkpoint
    ],
    verbose=1   # 학습되는 과정 보여주기
)

- training/validation의 loss, accuracy 확인

In [ ]:
history.history

---
# 6. Visualization

In [ ]:
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.plot(history['loss'], label='train')
plt.plot(history['val_loss'], label='val')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title("Loss")

plt.subplot(1, 2, 2)
plt.plot(history['accuracy'], label='train')
plt.plot(history['val_accuracy'], label='val')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.title("Accuracy")
plt.show()